# Finding causal genotype-phenotype relations with ddlite: extraction

## Introduction
In this example **ddlite** app, we'll build a system to indentify causal relationships between genotypes and phenotypes from raw journal articles. For an end-to-end example, see **GeneTaggerExample_Extraction.ipynb** and **GeneTaggerExample_Learning.ipynb**.

In [2]:
%load_ext autoreload
%autoreload 2

import cPickle, os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

data_dir = 'gene_phen_relation_example/{}'.format(os.environ.get('docs', 'data'))

from ddlite import *

## Processing the input data
We already downloaded the raw HTML for 2800 relevant article pages from PubMed. These can be found in the `data` folder. We can use ddlite's `DocParser` to read in the article text. It uses CoreNLP via ddlite's `SentenceParser` to parse each sentence. This can take a little while, so if the example has already been run, we'll reload it.

In [3]:
pkl_f = 'gene_phen_relation_example/gene_phen_saved_sents_v2.pkl'
try:
    with open(pkl_f, 'rb') as f:
        sents = cPickle.load(f)
except:
    print "Getting data from directory {}".format(data_dir)
    dp = DocParser(data_dir, TextParser())
    %time sents = dp.parseDocSentences()
    with open(pkl_f, 'w+') as f:
        cPickle.dump(sents, f)

print sents[0]

Getting data from directory gene_phen_relation_example/data
CPU times: user 24.7 s, sys: 1.21 s, total: 25.9 s
Wall time: 3min 32s
Sentence(words=[u'P-lacW', u'insertional', u'mutagenesis', u'on', u'the', u'second', u'chromosome', u'of', u'_', u'Drosophila', u'melanogaster', u'_', u':', u'isolation', u'of', u'lethals', u'with', u'different', u'overgrowth', u'phenotypes'], lemmas=[u'p-lacw', u'insertional', u'mutagenesis', u'on', u'the', u'second', u'chromosome', u'of', u'_', u'drosophila', u'melanogaster', u'_', u':', u'isolation', u'of', u'lethal', u'with', u'different', u'overgrowth', u'phenotype'], poses=[u'JJ', u'JJ', u'NN', u'IN', u'DT', u'JJ', u'NN', u'IN', u'CD', u'NN', u'FW', u'FW', u':', u'NN', u'IN', u'NNS', u'IN', u'JJ', u'NN', u'NNS'], dep_parents=[3, 3, 0, 7, 7, 7, 3, 12, 12, 12, 12, 7, 3, 3, 16, 14, 20, 20, 20, 14], dep_labels=[u'amod', u'amod', u'ROOT', u'case', u'det', u'amod', u'nmod', u'case', u'nummod', u'compound', u'compound', u'nmod', u'punct', u'dep', u'case', u'

## Extracting relation candidates with matchers
Extracting candidates for relations in ddlite is done with `Matcher` objects. Here, we'll use two `DictionaryMatcher`s. We have access to pretty comprehensive gene and phenotype dictionaries. Let's load them in and create the `DictionaryMatcher`s.

In [4]:
# Schema is: ENSEMBL_ID | NAME | TYPE (refseq, canonical, non-canonical)
genes = [line.rstrip().split('\t')[1] for line in open('gene_phen_relation_example/dicts/ensembl_genes.tsv')]
genes = filter(lambda g : len(g) > 3, genes)

# Schema is: HPO_ID | NAME | TYPE (exact, lemma)
phenos = [line.rstrip().split('\t')[1] for line in open('gene_phen_relation_example/dicts/pheno_terms.tsv')]

GM = DictionaryMatch(label='G', dictionary=genes, ignore_case=False)
PM = DictionaryMatch(label='P', dictionary=phenos)

If we wanted to define more `Matcher` for, say, genes, we could use multiple `Matcher` objects with a `MultiMatcher`. For now, we'll just use the single `DictionaryMatcher` for both classes. We'll use this to extract our candidate relations from the sentences into an `Relations` object. Using just the matchers will likely provide high recall but poor precision. This is because not all genotype-phenotype mention pairs in the same sentence represent a causal pairing. The `Relations` object we create can be used in a `CandidateModel`. This allows us to learn a model to predict whether each candidate pair represents a causal pair or not.

In [5]:
R = Relations(sents, GM, PM)

KeyboardInterrupt: 

We can visualize contexts for our extractions too. This may help in writing labeling functions in a learning task.

In [ ]:
R[2].render()

Finally, we can pickle the extracted candidates from our `Relations` object for later use.

In [ ]:
R.dump_candidates('gene_phen_relation_example/gene_phen_saved_relations_v3.pkl')